In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import bz2

In [2]:
pd.set_option('display.max_columns',None) # display all columns without compression

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
accepted_df = pd.read_csv('final_cleaned_accepted.csv',index_col='Unnamed: 0')
accepted_df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,deling_2yrs,fico_range_low,fico_range_high,ing_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_dealing,tot_coll_amt,tot_cur_bal,total_bal_il,max_bal_bc,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag,earliest_cr_line_year,earliest_cr_line_month,issue_d_year,issue_d_month,last_credit_pull_d_year,last_credit_pull_d_month,last_pymnt_d_year,last_pymnt_d_month
0,3600.0,3600.0,3600.0,36,13.99,123.03,C,10,1,55000.0,0,5,0,5.91,0.0,675.0,679.0,1.0,7.0,0.0,2765.0,29.7,13.0,1,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,122.67,564.0,560.0,0.0,1.0,0,0.0,722.0,144904.0,4981.0,722.0,9300.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,4.000000,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,0,0,0,2003,8,2015,12,2019,3,2019,1
1,24700.0,24700.0,24700.0,36,11.99,820.28,C,10,1,65000.0,0,5,0,16.06,1.0,715.0,719.0,4.0,22.0,0.0,21470.0,19.2,38.0,1,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,926.35,699.0,695.0,0.0,1.0,0,0.0,0.0,204396.0,18005.0,6472.0,111800.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,0.000000,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,0,0,0,1999,12,2015,12,2019,3,2016,6
2,20000.0,20000.0,20000.0,60,10.78,432.66,B,10,1,63000.0,0,5,0,10.78,0.0,695.0,699.0,0.0,6.0,0.0,7869.0,56.2,18.0,1,0.00,0.00,22705.924294,22705.92,20000.00,2705.92,0.0,0.0,0.0,15813.30,704.0,700.0,0.0,1.0,1,0.0,0.0,189699.0,10827.0,2081.0,14000.0,6.0,31617.0,2737.0,55.9,0.0,0.0,125.0,184.0,14.0,14.0,5.0,101.0,10.000000,0.0,2.0,3.0,2.0,4.0,6.0,4.0,7.0,3.0,6.0,0.0,0.0,0.0,0.0,100.0,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,0,0,0,2000,8,2015,12,2019,3,2017,6
3,35000.0,35000.0,35000.0,60,14.85,829.90,C,10,1,110000.0,1,1,0,17.06,0.0,785.0,789.0,0.0,13.0,0.0,7802.0,11.6,17.0,1,15897.65,15897.65,31464.010000,31464.01,19102.35,12361.66,0.0,0.0,0.0,829.90,679.0,675.0,0.0,1.0,0,0.0,0.0,301500.0,12609.0,6987.0,67300.0,2.0,23192.0,54962.0,12.1,0.0,0.0,36.0,87.0,2.0,2.0,1.0,2.0,7.084087,0.0,4.0,5.0,8.0,10.0,2.0,10.0,13.0,5.0,13.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,381215.0,52226.0,62500.0,18000.0,0,0,0,2008,9,2015,12,2019,3,2019,2
4,10400.0,10400.0,10400.0,60,22.45,289.91,F,3,1,104433.0,1,5,0,25.37,1.0,695.0,699.0,3.0,12.0,0.0,21929.0,64.5,35.0,1,0.00,0.00,11740.500000,11740.50,10400.00,1340.50,0.0,0.0,0.0,10128.96,704.0,700.0,0.0,1.0,0,0.0,0.0,331730.0,73839.0,9702.0,34000.0,10.0,27644.0,4567.0,77.5,0.0,0.0,128.0,210.0,4.0,4.0,6.0,4.0,1.000000,0.0,4.0,6.0,5.0,9.0,10.0,7.0,19.0,6.0,12.0,0.0,0.0,0.0,4.0,96.6,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,0,0,0,1998,6,2015,12,2018,3,2016,7


In [5]:
# seperating features and target columns

features = ['funded_amnt_inv',
     'int_rate',
     'emp_length',
     'annual_inc',
     'pymnt_plan',
     'dti',
     'deling_2yrs',
     'fico_range_low',
     'ing_last_6mths',
     'open_acc',
     'pub_rec',
     'revol_bal',
     'revol_util',
     'total_acc',
     'initial_list_status',
     'total_rec_late_fee',
     'last_pymnt_amnt',
     'last_fico_range_high',
     'last_fico_range_low',
     'collections_12_mths_ex_med',
     'policy_code',
     'application_type',
     'acc_now_dealing',
     'tot_coll_amt',
     'tot_cur_bal',
     'total_bal_il',
     'max_bal_bc',
     'issue_d_month',
     'earliest_cr_line_month',
     'earliest_cr_line_year',
     'last_pymnt_d_month',
     'last_credit_pull_d_month',
     'last_credit_pull_d_year']
X = accepted_df[features]
y = accepted_df['grade']

In [6]:
X.head()

,funded_amnt_inv,int_rate,emp_length,annual_inc,pymnt_plan,dti,deling_2yrs,fico_range_low,ing_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_rec_late_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_dealing,tot_coll_amt,tot_cur_bal,total_bal_il,max_bal_bc,issue_d_month,earliest_cr_line_month,earliest_cr_line_year,last_pymnt_d_month,last_credit_pull_d_month,last_credit_pull_d_year
0,3600.0,13.99,10,55000.0,0,5.91,0.0,675.0,1.0,7.0,0.0,2765.0,29.7,13.0,1,0.0,122.67,564.0,560.0,0.0,1.0,0,0.0,722.0,144904.0,4981.0,722.0,12,8,2003,1,3,2019
1,24700.0,11.99,10,65000.0,0,16.06,1.0,715.0,4.0,22.0,0.0,21470.0,19.2,38.0,1,0.0,926.35,699.0,695.0,0.0,1.0,0,0.0,0.0,204396.0,18005.0,6472.0,12,12,1999,6,3,2019
2,20000.0,10.78,10,63000.0,0,10.78,0.0,695.0,0.0,6.0,0.0,7869.0,56.2,18.0,1,0.0,15813.30,704.0,700.0,0.0,1.0,1,0.0,0.0,189699.0,10827.0,2081.0,12,8,2000,6,3,2019
3,35000.0,14.85,10,110000.0,0,17.06,0.0,785.0,0.0,13.0,0.0,7802.0,11.6,17.0,1,0.0,829.90,679.0,675.0,0.0,1.0,0,0.0,0.0,301500.0,12609.0,6987.0,12,9,2008,2,3,2019
4,10400.0,22.45,3,104433.0,0,25.37,1.0,695.0,3.0,12.0,0.0,21929.0,64.5,35.0,1,0.0,10128.96,704.0,700.0,0.0,1.0,0,0.0,0.0,331730.0,73839.0,9702.0,12,6,1998,7,3,2018


In [7]:
X.describe()

,funded_amnt_inv,int_rate,emp_length,annual_inc,pymnt_plan,dti,deling_2yrs,fico_range_low,ing_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_rec_late_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_dealing,tot_coll_amt,tot_cur_bal,total_bal_il,max_bal_bc,issue_d_month,earliest_cr_line_month,earliest_cr_line_year,last_pymnt_d_month,last_credit_pull_d_month,last_credit_pull_d_year
count,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2011813.0,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06,2.011813e+06
mean,1.524292e+04,1.303905e+01,6.211924e+00,7.886847e+04,2.713970e-04,1.922425e+01,3.085814e-01,6.989068e+02,5.429675e-01,1.184194e+01,1.998128e-01,1.658321e+04,4.994625e+01,2.464468e+01,7.015970e-01,1.501758e+00,3.497902e+03,6.889226e+02,6.773720e+02,1.718699e-02,1.0,5.569106e-02,4.302587e-03,2.397015e+02,1.447505e+05,3.679309e+04,5.819536e+03,6.765297e+00,6.739750e+00,1.999548e+03,4.998735e+00,4.307000e+00,2.018461e+03
std,9.218408e+03,4.854674e+00,3.732877e+00,1.088075e+05,1.647190e-02,1.442853e+01,8.673368e-01,3.309607e+01,8.378096e-01,5.632920e+00,5.782347e-01,2.200594e+04,2.449972e+01,1.192246e+01,4.575574e-01,1.182641e+01,6.091766e+03,7.227845e+01,1.085009e+02,1.432625e-01,0.0,2.293242e-01,7.092019e-02,8.858000e+03,1.593913e+05,3.549477e+04,4.467919e+03,3.412506e+00,3.376731e+00,7.767315e+00,3.214543e+00,2.872079e+00,9.520098e-01
min,7.250000e+02,5.310000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,6.600000e+02,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,-9.500000e-09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.933000e+03,1.000000e+00,1.000000e+00,2.012000e+03
25%,8.000000e+03,9.440000e+00,3.000000e+00,4.800000e+04,0.000000e+00,1.233000e+01,0.000000e+00,6.750000e+02,0.000000e+00,8.000000e+00,0.000000e+00,6.107000e+03,3.120000e+01,1.600000e+01,0.000000e+00,0.000000e+00,3.140600e+02,6.590000e+02,6.550000e+02,0.000000e+00,1.0,0.000000e+00,0.000000e+00,0.000000e+00,3.086100e+04,1.777000e+04,3.500000e+03,4.000000e+00,4.000000e+00,1.996000e+03,3.000000e+00,3.000000e+00,2.018000e+03
50%,1.312500e+04,1.262000e+01,7.000000e+00,6.580000e+04,0.000000e+00,1.819000e+01,0.000000e+00,6.900000e+02,0.000000e+00,1.100000e+01,0.000000e+00,1.149600e+04,4.980000e+01,2.300000e+01,1.000000e+00,0.000000e+00,6.090800e+02,6.990000e+02,6.950000e+02,0.000000e+00,1.0,0.000000e+00,0.000000e+00,0.000000e+00,8.323000e+04,3.679309e+04,5.819536e+03,7.000000e+00,7.000000e+00,2.001000e+03,3.000000e+00,3.000000e+00,2.019000e+03
75%,2.000000e+04,1.588000e+01,1.000000e+01,9.500000e+04,0.000000e+00,2.482000e+01,0.000000e+00,7.150000e+02,1.000000e+00,1.500000e+01,0.000000e+00,2.039800e+04,6.870000e+01,3.100000e+01,1.000000e+00,0.000000e+00,3.900520e+03,7.340000e+02,7.300000e+02,0.000000e+00,1.0,0.000000e+00,0.000000e+00,0.000000e+00,2.160970e+05,3.679309e+04,5.819536e+03,1.000000e+01,1.000000e+01,2.005000e+03,7.000000e+00,4.000000e+00,2.019000e+03
max,4.000000e+04,3.099000e+01,1.000000e+01,1.100000e+08,1.000000e+00,9.990000e+02,5.800000e+01,8.450000e+02,8.000000e+00,1.010000e+02,8.600000e+01,2.904836e+06,3.666000e+02,1.760000e+02,1.000000e+00,1.484340e+03,4.219205e+04,8.500000e+02,8.450000e+02,2.000000e+01,1.0,1.000000e+00,1.400000e+01,9.152545e+06,9.971659e+06,1.837038e+06,1.170668e+06,1.200000e+01,1.200000e+01,2.015000e+03,1.200000e+01,1.200000e+01,2.019000e+03


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,test_size=0.1)

In [7]:
rfc = RandomForestClassifier(bootstrap=True,random_state=9,warm_start=True,n_jobs=-1)
rfc.fit(X_train,y_train) # training the model
rfc.score(X_test,y_test) # predicting accuray on test set

0.9426539153602211

In [10]:
ss = StandardScaler()
X_train_ss = ss.fit_transform(X_train)

In [11]:
rfc_1 = RandomForestClassifier(bootstrap=True,random_state=9,warm_start=True,n_jobs=4)
rfc_1.fit(X_train_ss,y_train) # training the model

0.1400522909604239

In [13]:
X_test_ss = ss.transform(X_test)
rfc_1.score(X_test_ss,y_test) # predicting accuray on test set

0.9427185334672088

In [8]:
file = bz2.BZ2File('rfc_small_model.pkl','wb')
pickle.dump(rfc,file) # dumping the model